In [ ]:
from google.colab import drive
import sentencepiece as spm
import os
import sentencepiece as spm
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
drive.mount('/content/drive')

import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ================================
# TRANSFORMER CONFIG (COLAB CELL)
# ================================

# -------- Paths --------
DATA_DIR = "/content/drive/MyDrive/TransformerMT/DATA"
CHECKPOINT_DIR = "/content/drive/MyDrive/TransformerMT/checkpoints"

TRAIN_EN = f"{DATA_DIR}/train.en"
TRAIN_VI = f"{DATA_DIR}/train.vi"
VALID_EN = f"{DATA_DIR}/valid.en"
VALID_VI = f"{DATA_DIR}/valid.vi"

VOCAB_EN_PATH = f"{DATA_DIR}/vocab_en.pth"
VOCAB_VI_PATH = f"{DATA_DIR}/vocab_vi.pth"

# -------- Special Tokens --------
PAD_TOKEN = "<pad>"
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"
UNK_TOKEN = "<unk>"

PAD_IDX = 0
BOS_IDX = 1
EOS_IDX = 2
UNK_IDX = 3

# -------- Model Hyperparameters --------
D_MODEL = 512
N_HEADS = 8
N_LAYERS = 6
D_FF = 2048
DROPOUT_RATE = 0.1

# -------- Training Hyperparameters --------
BATCH_SIZE = 64
EPOCHS = 150

LR = 1e-4
BETAS = (0.9, 0.98)
EPS = 1e-9

LABEL_SMOOTHING = 0.1
GRAD_CLIP = 1.0
WARMUP_STEPS = 4000

# -------- Sequence --------
MAX_LEN = 128

# -------- Device --------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -------- Logging / Saving --------
LOG_INTERVAL = 100
SAVE_EVERY_EPOCH = 1

VOCAB_SIZE_SPM=10000
MODEL_TYPE = 'bpe'

BATCH_SIZE = 64
MAX_SEQ_LEN = 128
VAL_SPLIT_RATIO = 0.1
SEED = 42
TEST_MODE_LIMIT=None
MAX_LEN_DECODE = 128
BEAM_SIZE = 5
TEST_EN_PATH = os.path.join(DATA_DIR, 'tst2013.en')
TEST_VI_PATH = os.path.join(DATA_DIR, 'tst2013.vi')
METRIC_LOG_PATH = os.path.join(DATA_DIR, 'training_log.json')





In [ ]:
import os
import torch
import sentencepiece as spm
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from google.colab import drive


# ==============================================================================
# 2. CÁC HÀM TIỆN ÍCH (LOAD & STATS)
# ==============================================================================

def load_raw_data(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Không tìm thấy file: {file_path}")
    print(f"-> Đang đọc file: {os.path.basename(file_path)}...")
    with open(file_path, encoding='utf-8') as f:
        return [line.strip() for line in f]

def plot_statistics(raw_data, sp_processor, lang_name):
    """
    Thống kê độ dài câu dựa trên số lượng Subword (Token IDs).
    Giúp quyết định MAX_SEQ_LEN hợp lý.
    """
    # Lấy mẫu tối đa 50k câu để thống kê cho nhanh
    sample_data = raw_data[:50000] if len(raw_data) > 50000 else raw_data

    # Tokenize để đếm số lượng subword thực tế
    lengths = [len(sp_processor.encode_as_ids(s.lower())) for s in sample_data]

    if not lengths:
        print(f"Dữ liệu {lang_name} trống!")
        return

    print(f"\n[Thống kê {lang_name} (trên {len(sample_data)} mẫu)]")
    print(f"- Độ dài trung bình: {np.mean(lengths):.2f} subwords")
    print(f"- Max length: {np.max(lengths)} subwords")
    print(f"- 95% dữ liệu ngắn hơn: {np.percentile(lengths, 95):.0f} subwords")
    print(f"- 99% dữ liệu ngắn hơn: {np.percentile(lengths, 99):.0f} subwords")

    # Vẽ biểu đồ
    plt.figure(figsize=(8, 4))
    plt.hist(lengths, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    plt.title(f"Phân phối độ dài Subword ({lang_name})")
    plt.xlabel("Số lượng Subword")
    plt.ylabel("Số lượng câu")
    plt.grid(axis='y', alpha=0.3)
    plt.show()

# ==============================================================================
# 3. DATASET & COLLATE FUNCTION
# ==============================================================================

class TranslationDatasetSPM(Dataset):
    """
    Dataset thực hiện:
    1. Nhận raw text.
    2. Tokenize bằng SentencePiece.
    3. Lọc bỏ câu quá dài (theo Subword Count).
    4. Thêm <sos> và <eos>.
    """
    def __init__(self, raw_en, raw_vi, sp_en, sp_vi, max_len=100):
        self.sp_en = sp_en
        self.sp_vi = sp_vi
        self.SOS_IDX = sp_en.bos_id()
        self.EOS_IDX = sp_en.eos_id()

        self.data = []
        original_count = len(raw_en)

        print(f"-> Đang xử lý và lọc {original_count} cặp câu (Max Len={max_len})...")

        # Duyệt qua từng cặp câu
        for en, vi in zip(raw_en, raw_vi):
            # 1. Encode sang IDs (lowercase để chuẩn hóa)
            en_ids = self.sp_en.encode_as_ids(en.lower())
            vi_ids = self.sp_vi.encode_as_ids(vi.lower())

            # 2. Kiểm tra độ dài (Tính cả SOS và EOS sẽ thêm vào)
            if len(en_ids) + 2 <= max_len and len(vi_ids) + 2 <= max_len:
                self.data.append((en_ids, vi_ids))

        filtered_count = len(self.data)
        removed_count = original_count - filtered_count
        print(f"-> Hoàn tất. Giữ lại: {filtered_count} | Loại bỏ: {removed_count} ({removed_count/original_count:.2%})")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        en_ids, vi_ids = self.data[index]

        # Thêm SOS đầu và EOS cuối
        en_out = [self.SOS_IDX] + en_ids + [self.EOS_IDX]
        vi_out = [self.SOS_IDX] + vi_ids + [self.EOS_IDX]

        # Trả về LongTensor
        return torch.tensor(en_out, dtype=torch.long), torch.tensor(vi_out, dtype=torch.long)

class MyCollateSPM:
    """
    Hàm gom batch: Padding các câu trong batch cho bằng nhau.
    """
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        # Tách src và trg từ batch list
        src = [item[0] for item in batch]
        trg = [item[1] for item in batch]

        # Padding (batch_first=True -> Output: (Batch, Seq_Len))
        src = pad_sequence(src, batch_first=True, padding_value=self.pad_idx)
        trg = pad_sequence(trg, batch_first=True, padding_value=self.pad_idx)

        return src, trg

# ==============================================================================
# 4. LUỒNG CHẠY CHÍNH (MAIN DATA PIPELINE)
# ==============================================================================

def process_data_pipeline():
    print("=== BẮT ĐẦU QUY TRÌNH XỬ LÝ DỮ LIỆU ===\n")

    # -----------------------------------------
    # BƯỚC 1: LOAD DỮ LIỆU THÔ
    # -----------------------------------------
    path_en = os.path.join(DATA_DIR, 'train.en')
    path_vi = os.path.join(DATA_DIR, 'train.vi')

    raw_en = load_raw_data(path_en)
    raw_vi = load_raw_data(path_vi)

    assert len(raw_en) == len(raw_vi), "Lỗi: Số lượng câu Anh-Việt không khớp!"
    print(f"-> Tổng số câu raw: {len(raw_en)}")

    # -----------------------------------------
    # BƯỚC 2: CHIA TẬP TRAIN / VAL
    # -----------------------------------------
    print(f"\n--- Chia tập dữ liệu (Val Ratio: {VAL_SPLIT_RATIO}) ---")
    en_train, en_val, vi_train, vi_val = train_test_split(
        raw_en, raw_vi, test_size=VAL_SPLIT_RATIO, random_state=SEED
    )
    print(f"-> Train set: {len(en_train)} câu")
    print(f"-> Val set:   {len(en_val)} câu")

    # -----------------------------------------
    # BƯỚC 3: LOAD MODEL SENTENCEPIECE (ĐÃ TRAIN SẴN)
    # -----------------------------------------
    print("\n--- Load SentencePiece Models ---")
    # Giả định file model tên là spm_en.model và spm_vi.model
    sp_en_path = os.path.join(DATA_DIR, 'spm_en.model')
    sp_vi_path = os.path.join(DATA_DIR, 'spm_vi.model')

    if not os.path.exists(sp_en_path) or not os.path.exists(sp_vi_path):
        raise FileNotFoundError("Chưa tìm thấy file model SPM (.model). Hãy đảm bảo bạn đã train SPM trước đó.")

    sp_en = spm.SentencePieceProcessor()
    sp_en.load(sp_en_path)

    sp_vi = spm.SentencePieceProcessor()
    sp_vi.load(sp_vi_path)

    PAD_IDX = sp_en.pad_id() # Thường là 0
    print(f"-> Đã load SPM. Vocab EN: {sp_en.get_piece_size()} | Vocab VI: {sp_vi.get_piece_size()}")
    print(f"-> PAD_IDX: {PAD_IDX}")

    # -----------------------------------------
    # BƯỚC 4: THỐNG KÊ (Dựa trên Subword)
    # -----------------------------------------
    print("\n--- Thống kê Dữ liệu Train (Subword Level) ---")
    plot_statistics(en_train, sp_en, "English (Source)")
    plot_statistics(vi_train, sp_vi, "Vietnamese (Target)")

    # -----------------------------------------
    # BƯỚC 5: TẠO DATASET & DATALOADER
    # -----------------------------------------
    print("\n--- Tạo Dataset và DataLoader ---")

    # Tạo Dataset (Tokenize & Filter)
    train_dataset = TranslationDatasetSPM(en_train, vi_train, sp_en, sp_vi, max_len=MAX_SEQ_LEN)
    val_dataset = TranslationDatasetSPM(en_val, vi_val, sp_en, sp_vi, max_len=MAX_SEQ_LEN)

    # Collate Function
    collate_fn = MyCollateSPM(pad_idx=PAD_IDX)

    # Tạo DataLoader
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=2,    # Tăng tốc độ load data (tuỳ chọn)
        pin_memory=True   # Tối ưu khi chuyển sang GPU
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=2,
        pin_memory=True
    )

    print(f"\n-> Train Loader: {len(train_loader)} batches")
    print(f"-> Val Loader:   {len(val_loader)} batches")

    # Kiểm tra thử 1 batch
    src_batch, trg_batch = next(iter(train_loader))
    print(f"\n[Test Batch Shape]")
    print(f"Src: {src_batch.shape} (Batch, Seq_len)")
    print(f"Trg: {trg_batch.shape}")

    return train_loader, val_loader, sp_en, sp_vi

# --- CHẠY CHƯƠNG TRÌNH ---
if __name__ == "__main__":
    #train_dl, val_dl, sp_en, sp_vi = process_data_pipeline()
    print("\n>>> XỬ LÝ DỮ LIỆU HOÀN TẤT. SẴN SÀNG TRAIN MODEL <<<")


>>> XỬ LÝ DỮ LIỆU HOÀN TẤT. SẴN SÀNG TRAIN MODEL <<<


In [ ]:
sp_en = spm.SentencePieceProcessor()
sp_en.load(os.path.join(DATA_DIR, 'spm_en.model'))

sp_vi = spm.SentencePieceProcessor()
sp_vi.load(os.path.join(DATA_DIR, 'spm_vi.model'))

True

# Scaled Dot-Product Attention & Multi-Head Attention

In [ ]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F


def scaled_dot_product_attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)

    # scores shape: (Batch_size, Num_Heads, Seq_len_Q, Seq_len_K)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    # 1. Áp dụng Mask
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)

    # 2. Tính Softmax
    p_attn = torch.softmax(scores, dim=-1)

    # 3. ÁP DỤNG DROPOUT
    if dropout is not None:
        p_attn = dropout(p_attn)

    # 4. Tính đầu ra
    output = torch.matmul(p_attn, value)

    return output, p_attn

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout_rate=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model phải chia hết cho num_heads"

        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        self.d_model = d_model

        # 4 lớp Linear: Wq, Wk, Wv, và Wo (output)
        self.w_q = nn.Linear(d_model, d_model, bias=False)
        self.w_k = nn.Linear(d_model, d_model, bias=False)
        self.w_v = nn.Linear(d_model, d_model, bias=False)
        self.w_o = nn.Linear(d_model, d_model, bias=False)

        # Dropout Module
        self.dropout = nn.Dropout(p=dropout_rate)


    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        if mask is not None and mask.dim() == 3:
            # (B, L, L) → (B, 1, L, L)
            mask = mask.unsqueeze(1)

        # 1. Ánh xạ tuyến tính và tách thành Heads
        def transform(x, w):
            return w(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        q = transform(query, self.w_q)
        k = transform(key, self.w_k)
        v = transform(value, self.w_v)

        # 2. Tính Scaled Dot-Product Attention (truyền module dropout vào)
        # Bắt buộc phải truyền self.dropout để gọi hàm forward() của nn.Dropout
        x, self_attn = scaled_dot_product_attention(q, k, v, mask=mask, dropout=self.dropout)

        # 3. Nối các đầu (Concatenate Heads)
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        # 4. Ánh xạ tuyến tính cuối cùng (Wo)
        return self.w_o(x), self_attn

# Positional Encoding (Sinusoidal):

In [ ]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    """
    Positional Encoding (PE) Sinusoidal theo công thức gốc của bài báo "Attention Is All You Need".
    PE là một ma trận cố định, không học được.
    """
    def __init__(self, d_model, dropout_rate, max_len=5000):
        super(PositionalEncoding, self).__init__()

        self.dropout = nn.Dropout(p=dropout_rate)

        # Tạo ma trận PE (Max_Len, D_model)
        pe = torch.zeros(max_len, d_model)

        # 1. Tính Pos (Vị trí): (Max_Len, 1)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # 2. Tính Div_term (Mẫu số): 10000^(2i/d_model)
        # Sử dụng log và exp để tránh lỗi số học: exp(2i * -log(10000) / d_model)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )

        # 3. Gán giá trị Sin/Cos
        # Cột chẵn (2i) là Sin:
        pe[:, 0::2] = torch.sin(position * div_term)

        # Cột lẻ (2i+1) là Cos:
        # Nếu d_model lẻ, cột cuối cùng sẽ được gán bằng pe[:, d_model-1]
        pe[:, 1::2] = torch.cos(position * div_term)

        # Thêm chiều batch (1, Max_Len, D_model)
        pe = pe.unsqueeze(0)

        # Đăng ký là buffer (không tham gia vào quá trình backward/gradient descent)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Input: x (Embedding đầu vào) shape: (Batch_size, Seq_len, D_model)
        Output: x + PE
        """
        # Thêm Positional Encoding vào Embedding.
        # Ta chỉ lấy PE đến độ dài Seq_len của batch hiện tại.
        # Kích thước của x là x.size(1)
        x = x + self.pe[:, :x.size(1)]

        # Áp dụng Dropout
        return self.dropout(x)



# Transformer Encoder Layer

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class PositionwiseFeedForward(nn.Module):
    """
    Lớp Feed-Forward Network (FFN) trong Transformer.
    Áp dụng riêng rẽ (position-wise) cho từng vị trí trong chuỗi.
    """
    def __init__(self, d_model, d_ff, dropout_rate=0.1):
        super(PositionwiseFeedForward, self).__init__()

        # Lớp 1: Tăng chiều (d_model -> d_ff)
        self.w_1 = nn.Linear(d_model, d_ff)

        # Lớp 2: Giảm chiều (d_ff -> d_model)
        self.w_2 = nn.Linear(d_ff, d_model)

        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        # x shape: (Batch_size, Seq_len, D_model)

        # 1. Linear 1 -> ReLU
        # Output shape: (Batch_size, Seq_len, D_ff)
        output = F.relu(self.w_1(x))

        # 2. Dropout
        output = self.dropout(output)

        # 3. Linear 2
        # Output shape: (Batch_size, Seq_len, D_model)
        return self.w_2(output)

In [ ]:
import torch

class SublayerConnection(nn.Module):
    def __init__(self, size, dropout_rate=0.1):
        super(SublayerConnection, self).__init__()

        # size chính là d_model (ví dụ: 512)
        self.norm = nn.LayerNorm(size)
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x, sublayer):
        """
        Input: x (input của sublayer), sublayer (hàm/module của sublayer: MHA hoặc FFN)
        """
        # 1. Layer Normalization
        norm_x = self.norm(x)

        # 2. Tính Sublayer (MHA hoặc FFN)
        sublayer_output = sublayer(norm_x)

        # 3. Dropout
        sublayer_output = self.dropout(sublayer_output)

        # 4. Residual Connection (Add)
        return x + sublayer_output


In [ ]:
class EncoderLayer(nn.Module):
    """
    Một lớp đơn lẻ trong Transformer Encoder.
    """
    def __init__(self, size, self_attn, feed_forward, dropout_rate):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn        # MultiHeadAttention
        self.feed_forward = feed_forward  # PositionwiseFeedForward
        self.sublayer = nn.ModuleList([
            SublayerConnection(size, dropout_rate), # Cho Self-Attention
            SublayerConnection(size, dropout_rate)  # Cho FFN
        ])
        self.size = size # d_model

    def forward(self, x, mask):
        # x shape: (Batch_size, Seq_len, D_model)

        # 1. Sublayer 1: Multi-Head Self-Attention (với Residual & Norm)
        # Tự chú ý (Self-Attention): Q=K=V=x. Encoder không cần look-ahead mask.
        # Mask ở đây là Padding Mask (Mask: 1 là giữ, 0 là ẩn)
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask=mask)[0])

        # 2. Sublayer 2: Feed-Forward Network (với Residual & Norm)
        x = self.sublayer[1](x, self.feed_forward)

        return x

# Transformer Decoder Layer

In [ ]:
import torch

def subsequent_mask(size):
    return torch.tril(torch.ones(1, size, size, dtype=torch.bool))

def make_std_mask(tgt, pad_idx):
    """
    Tạo Target Mask (kết hợp Padding Mask và Look-Ahead Mask).
    Dùng cho Decoder Self-Attention.
    """
    # 1. Tạo Padding Mask (True/1 cho vị trí KHÔNG phải PAD)
    # unsqueeze(-2) để kích thước là (B, 1, L_tgt)
    tgt_pad_mask = (tgt != pad_idx).unsqueeze(-2)

    # 2. Tạo Look-Ahead Mask
    seq_len = tgt.size(-1)
    look_ahead_mask = subsequent_mask(seq_len).type_as(tgt_pad_mask.data)

    # 3. Kết hợp (AND logic): Chỉ True nếu KHÔNG phải PAD VÀ KHÔNG phải tương lai
    tgt_mask = tgt_pad_mask & look_ahead_mask

    return tgt_mask.requires_grad_(False)

# =================================================================
# PHẦN B: LỚP BATCH - TỔ CHỨC INPUT VÀ TẠO MASKS CHO MỖI LẦN CHẠY
# =================================================================

class Batch:
    """
    Lớp đóng gói dữ liệu đầu vào (src/tgt) và tạo ra tất cả các masks cần thiết.
    """
    def __init__(self, src, tgt, pad_idx):
        self.src = src # (B, L_src)

        # 1. Tách Target cho Input và Output
        # TGT INPUT: Dùng để đưa vào Decoder (gồm <sos>...tok_n)
        self.tgt = tgt[:, :-1]
        # TGT OUTPUT (tgt_y): Dùng để tính Loss (gồm tok_1...<eos>)
        self.tgt_y = tgt[:, 1:]

        # 2. TẠO MASKS CẦN THIẾT

        # a) Source Mask (Padding Mask): Dùng cho ENCODER & CROSS-ATTENTION
        # Kích thước (B, 1, L_src)
        self.src_mask = (src != pad_idx).unsqueeze(-2).requires_grad_(False)

        # b) Target Mask (Padding + Look-Ahead): Dùng cho DECODER SELF-ATTENTION
        # Kích thước (B, L_tgt, L_tgt)
        self.tgt_mask = make_std_mask(self.tgt, pad_idx)

        # Số lượng token thực tế (không phải pad) để chuẩn hóa Loss
        self.ntokens = (self.tgt_y != pad_idx).data.sum()


In [ ]:
class DecoderLayer(nn.Module):
    """
    Một lớp đơn lẻ trong Transformer Decoder.
    Chứa: Masked Self-Attention, Cross-Attention, và FFN.
    """
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout_rate):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn      # MultiHeadAttention (Cho Self-Attention)
        self.src_attn = src_attn        # MultiHeadAttention (Cho Cross-Attention)
        self.feed_forward = feed_forward

        # 3 Sublayer Connections cho 3 Sublayer
        self.sublayer = nn.ModuleList([
            SublayerConnection(size, dropout_rate), # Masked Self-Attention
            SublayerConnection(size, dropout_rate), # Cross-Attention
            SublayerConnection(size, dropout_rate)  # FFN
        ])

    def forward(self, x, memory, src_mask, tgt_mask):
        """
        Input:
          x: decoder input (output của lớp trước) (Batch_size, L_tgt, D_model)
          memory: output của Encoder (Batch_size, L_src, D_model)
          src_mask: Padding Mask cho Encoder Output (memory)
          tgt_mask: Look-Ahead Mask (Subsequent Mask) cho Decoder Input (x)
        """

        # 1. Sublayer 1: Masked Multi-Head Self-Attention
        # Q=K=V=x. Dùng tgt_mask (Look-Ahead Mask)
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask=tgt_mask)[0])

        # 2. Sublayer 2: Multi-Head Cross-Attention (Encoder-Decoder Attention)
        # Q=x (target), K=V=memory (source/encoder output)
        # Dùng src_mask (Padding Mask) để không chú ý đến các token <pad> từ Encoder
        x = self.sublayer[1](x, lambda x: self.src_attn(x, memory, memory, mask=src_mask)[0])

        # 3. Sublayer 3: Feed-Forward Network
        x = self.sublayer[2](x, self.feed_forward)

        return x

# Transformer

In [ ]:
class Encoder(nn.Module):
    """
    Toàn bộ Transformer Encoder, bao gồm Stacking N Encoder Layers.
    """
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        # layer: là một đối tượng EncoderLayer đã được khởi tạo
        # N: số lượng lớp (ví dụ N=6)

        # Stacking N lớp
        self.layers = clones(layer, N)

        # Layer Normalization cuối cùng
        self.norm = nn.LayerNorm(layer.size)

        self.size = layer.size

    def forward(self, x, mask):
        # x shape: (Batch_size, Seq_len, D_model) - Đã là Embedding + PE
        # mask: Padding Mask (từ Source)

        # Chạy qua N lớp Encoder Layer
        for layer in self.layers:
            x = layer(x, mask)

        # Áp dụng Layer Norm cuối cùng trước khi trả về
        return self.norm(x)

In [ ]:
class Decoder(nn.Module):
    """
    Toàn bộ Transformer Decoder, bao gồm Stacking N Decoder Layers.
    """
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        # layer: là một đối tượng DecoderLayer đã được khởi tạo

        # Stacking N lớp
        self.layers = clones(layer, N)

        # Layer Normalization cuối cùng
        self.norm = nn.LayerNorm(layer.size)

        self.size = layer.size

    def forward(self, x, memory, src_mask, tgt_mask):
        # x: Decoder Input (Embedding + PE)
        # memory: Output của Encoder (Encoder Memory)
        # src_mask: Padding Mask của Source
        # tgt_mask: Look-Ahead Mask của Target

        # Chạy qua N lớp Decoder Layer
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)

        # Áp dụng Layer Norm cuối cùng trước khi trả về
        return self.norm(x)

In [ ]:
import copy

class Transformer(nn.Module):
    """
    Mô hình Transformer hoàn chỉnh (Source-to-Target)
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(Transformer, self).__init__()
        self.encoder = encoder      # Lớp Encoder (Stacking N Encoder Layers)
        self.decoder = decoder      # Lớp Decoder (Stacking N Decoder Layers)
        self.src_embed = src_embed  # Lớp Embedding + PE cho Source
        self.tgt_embed = tgt_embed  # Lớp Embedding + PE cho Target
        self.generator = generator  # Lớp Linear cuối cùng (Projection to Vocab)

    def forward(self, src, tgt, src_mask, tgt_mask):
        """
        Input:
          src: Source sequence (indices) (B, L_src)
          tgt: Target sequence (indices) (B, L_tgt)
          src_mask: Padding Mask (B, 1, L_src)
          tgt_mask: Look-Ahead Mask (B, L_tgt, L_tgt)
        """

        # 1. Chạy Encoder và lấy Memory
        memory = self.encode(src, src_mask) # (B, L_src, D_model)

        # 2. Chạy Decoder
        output = self.decode(memory, src_mask, tgt, tgt_mask) # (B, L_tgt, D_model)

        # 3. Projection Logits (Chuyển D_model thành Vocab Size)
        return self.generator(output)

    def encode(self, src, src_mask):
        # src_embed(src): Embedding + Positional Encoding
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        # tgt_embed(tgt): Embedding + Positional Encoding
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

# --- Generator/Projection Layer ---
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab_size):
        super(Generator, self).__init__()
        # Lớp tuyến tính ánh xạ D_model về Vocab Size
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # Trả về Logits (trước Softmax)
        return self.proj(x)

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
# --- Hàm utility clones (rất quan trọng) ---
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


def make_model(src_vocab, tgt_vocab, N=N_LAYERS, d_model=D_MODEL, d_ff=D_FF, h=N_HEADS, dropout=DROPOUT_RATE):
    "Hàm xây dựng mô hình Transformer từ các khối đã định nghĩa."

    # 1. Khởi tạo các Sub-module
    c = copy.deepcopy

    # Giả định các lớp này đã được định nghĩa chi tiết (MHA, FFN, PE, SublayerConnection)
    attn = MultiHeadAttention(d_model, h, dropout)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    pe = PositionalEncoding(d_model, dropout)
    sublayer_conn = SublayerConnection(d_model, dropout) # Mẫu để clones

    # 2. Xây dựng Layer mẫu (sử dụng clones an toàn cho Sublayers)
    class SafeEncoderLayer(EncoderLayer):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.sublayer = clones(sublayer_conn, 2)

    class SafeDecoderLayer(DecoderLayer):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.sublayer = clones(sublayer_conn, 3)

    enc_layer_proto = SafeEncoderLayer(d_model, c(attn), c(ff), dropout)
    dec_layer_proto = SafeDecoderLayer(d_model, c(attn), c(attn), c(ff), dropout)

    # 3. Lắp ráp Encoder/Decoder lớn (dùng clones cho N layers)
    encoder = Encoder(enc_layer_proto, N)
    decoder = Decoder(dec_layer_proto, N)

    # 4. Lớp Embedding và Generator
    src_embed = nn.Sequential(Embeddings(d_model, src_vocab), c(pe))
    tgt_embed = nn.Sequential(Embeddings(d_model, tgt_vocab), c(pe))
    generator = Generator(d_model, tgt_vocab)

    # 5. Mô hình Hoàn chỉnh
    model = Transformer(
        encoder, decoder,
        src_embed, tgt_embed,
        generator
    )

    # 6. Khởi tạo tham số bằng Xavier (Giúp ổn định training)
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return model



# Training Loop

In [ ]:
from torch.nn.utils import clip_grad_norm_

class LossCompute:
    def __init__(self, criterion, opt=None, clip_norm=None):
        self.criterion = criterion
        self.opt = opt
        self.clip_norm = clip_norm

    def __call__(self, x, y, norm):
        loss = self.criterion(
            x.contiguous().view(-1, x.size(-1)),
            y.contiguous().view(-1)
        )

        normalized_loss = loss / norm

        if self.opt is not None:
            normalized_loss.backward()

            if self.clip_norm is not None:
                clip_grad_norm_(
                    (p for g in self.opt.optimizer.param_groups for p in g['params']),
                    self.clip_norm
                )

            self.opt.step()
            self.opt.zero_grad()

        return loss


In [ ]:
class LabelSmoothing(nn.Module):
    def __init__(self, size, padding_idx, smoothing=0.1):
        super().__init__()
        self.size = size
        self.padding_idx = padding_idx
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing
        self.denominator = size - 2
        self.criterion = nn.KLDivLoss(reduction='sum')

    def forward(self, logits, target):
        assert logits.size(1) == self.size

        log_probs = F.log_softmax(logits, dim=-1)

        with torch.no_grad():
            true_dist = torch.zeros_like(log_probs)
            true_dist.fill_(self.smoothing / self.denominator)
            true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
            true_dist[:, self.padding_idx] = 0.0
            true_dist[target == self.padding_idx] = 0.0

        return self.criterion(log_probs, true_dist)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

class NoamOpt:
    "Optimizer tùy chỉnh với Noam/Warmup scheduler, hỗ trợ Checkpointing."
    def __init__(self, model_size, warmup_steps, optimizer):
        # Lưu trữ base optimizer (có thể là Adam, AdamW,...)
        self.optimizer = optimizer

        # Các trạng thái cần lưu
        self._step = 0
        self.warmup_steps = warmup_steps
        self.model_size = model_size
        self._rate = 0

        # Thiết lập LR ban đầu trong param_groups
        self.set_lr_in_param_groups(0)

    def set_lr_in_param_groups(self, rate):
        "Hàm tiện ích để cập nhật LR cho base optimizer."
        for p in self.optimizer.param_groups:
            p['lr'] = rate

    def step(self):
        "Cập nhật các tham số và tốc độ học."
        self._step += 1
        rate = self.rate()

        # Cập nhật LR của base optimizer
        self.set_lr_in_param_groups(rate)

        # Gọi step của base optimizer
        self.optimizer.step()

    def rate(self, step=None):
        "Tính toán LR theo công thức Noam."
        if step is None:
            step = self._step
        if step == 0:
            return 0

        # Công thức Noam: d_model^(-0.5) * min(step^(-0.5), step * warmup_steps^(-1.5))
        return self.model_size**(-0.5) * \
               min(step**(-0.5), step * self.warmup_steps**(-1.5))

    def zero_grad(self):
        "Gọi zero_grad của base optimizer."
        self.optimizer.zero_grad()

    # --- PHƯƠNG THỨC HỖ TRỢ CHECKPOINTING (Vấn đề 2) ---
    def state_dict(self):
        "Trả về state_dict của Scheduler (bao gồm trạng thái của base optimizer và step)."
        return {
            'optimizer': self.optimizer.state_dict(),
            'step': self._step,
            'warmup_steps': self.warmup_steps,
            'model_size': self.model_size
        }

    def load_state_dict(self, state_dict):
        "Load trạng thái cho Scheduler."
        self.optimizer.load_state_dict(state_dict['optimizer'])
        self._step = state_dict['step']
        self.warmup_steps = state_dict['warmup_steps']
        self.model_size = state_dict['model_size']
        # Sau khi load, cần cập nhật lại LR hiện tại
        self.set_lr_in_param_groups(self.rate())


In [ ]:

def calculate_ppl(val_loader, model, pad_idx, criterion_val):
    model.eval()
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for src_batch, tgt_batch in val_loader:
            src_batch, tgt_batch = src_batch.to(DEVICE), tgt_batch.to(DEVICE)
            batch = Batch(src_batch, tgt_batch, pad_idx)

            output_logits = model(
                batch.src, batch.tgt,
                batch.src_mask, batch.tgt_mask
            )

            loss = criterion_val(
                output_logits.view(-1, output_logits.size(-1)),
                batch.tgt_y.view(-1)
            )

            total_loss += loss.item()
            total_tokens += batch.ntokens.item()

    if total_tokens == 0:
        return float('inf'), float('inf')

    avg_loss = total_loss / total_tokens
    ppl = math.exp(avg_loss)
    return avg_loss, ppl


In [ ]:
import torch.nn.functional as F

# Hàm này sẽ được gọi từ bên trong run_full_training_pipeline
def calculate_ppl(val_loader, model, pad_idx, criterion_val):
    "Tính Perplexity trên Validation Set bằng Cross-Entropy Loss chuẩn."
    model.eval() # Bắt buộc phải chuyển sang chế độ đánh giá
    total_loss = 0
    total_tokens = 0

    with torch.no_grad():
        for src_batch, tgt_batch in val_loader:
            src_batch, tgt_batch = src_batch.to(DEVICE), tgt_batch.to(DEVICE)
            batch = Batch(src_batch, tgt_batch, pad_idx)

            output_logits = model(batch.src, batch.tgt, batch.src_mask, batch.tgt_mask)

            # Log-Softmax Logits (Vì criterion_val là CrossEntropyLoss)
            log_probs = F.log_softmax(output_logits, dim=-1)

            # Tính Loss bằng Cross-Entropy chuẩn (reduction='sum')
            loss = criterion_val(log_probs.contiguous().view(-1, output_logits.size(-1)),
                                 batch.tgt_y.contiguous().view(-1))

            total_loss += loss.item()
            total_tokens += batch.ntokens.item()

    if total_tokens == 0:
        return float('inf'), float('inf')

    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    return avg_loss, perplexity

In [ ]:
import torch
import os
import math
from tqdm import tqdm
from torch.nn.utils import clip_grad_norm_

# --- CÁC THAM SỐ CHUNG VÀ BIẾN GLOBAL (Giả định) ---
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# PAD_IDX = 0
# N_EPOCHS = 10
# CHECKPOINT_DIR = '/content/drive/MyDrive/TransformerMT/checkpoints'
# Giả định: model, train_loader, val_loader, criterion, optimizer đã được khởi tạo

# --- CÁC HÀM TIỆN ÍCH CẦN THIẾT ---
# Batch, calculate_ppl, LossCompute, NoamOpt đã được định nghĩa và sửa đổi

def save_checkpoint(model, optimizer, epoch, best_val_ppl,
                    is_best=False, filename='latest_checkpoint.pth'):

    os.makedirs(CHECKPOINT_DIR, exist_ok=True)

    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_val_ppl': best_val_ppl,
        'config': {
            'pad_idx': PAD_IDX,
            'n_epochs': EPOCHS,
            'clip_norm': GRAD_CLIP
        }
    }

    path = os.path.join(CHECKPOINT_DIR, filename)
    torch.save(state, path)

    if is_best:
        torch.save(state, os.path.join(CHECKPOINT_DIR, 'best_model.pth'))
        print(f"🔥 Best model saved at epoch {epoch}")

    print(f"💾 Checkpoint saved: {path}")


def load_checkpoint(model, optimizer, filename='latest_checkpoint.pth'):
    """Tải trạng thái mô hình và optimizer để tiếp tục training."""
    filepath = os.path.join(CHECKPOINT_DIR, filename)

    if not os.path.exists(filepath):
        print(f"Không tìm thấy checkpoint tại {filepath}. Bắt đầu training từ đầu.")
        return 1, float('inf') # Trả về epoch = 1 và PPL vô cực

    print(f"-> Đang tải checkpoint từ {filepath}...")

    checkpoint = torch.load(filepath, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_val_ppl = checkpoint['best_val_ppl']

    print(f"-> Tải thành công. Tiếp tục training từ Epoch {start_epoch}, PPL tốt nhất: {best_val_ppl:.2f}")

    return start_epoch, best_val_ppl


def run_full_training_pipeline(model, train_loader, val_loader,
                               criterion_train, criterion_val, # <-- ĐÃ SỬA CHỮ KÝ
                               optimizer, pad_idx, n_epochs, clip_norm=1.0):

    # 1. TẢI CHECKPOINT
    start_epoch, best_val_ppl = load_checkpoint(model, optimizer)

    # Khởi tạo Loss Compute (Sử dụng criterion_train cho Forward/Backward)
    loss_compute = LossCompute(criterion_train, optimizer, clip_norm=clip_norm)

    for epoch in range(start_epoch, n_epochs + 1):

        model.train()

        # Tích lũy Loss và Tokens dưới dạng Tensor trên GPU (Để tối ưu hiệu năng)
        total_loss = torch.tensor(0.0, device=DEVICE)
        total_tokens = torch.tensor(0, device=DEVICE)

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{n_epochs} (Train)", unit="batch")

        for i, (src_batch, tgt_batch) in enumerate(progress_bar):

            src_batch, tgt_batch = src_batch.to(DEVICE), tgt_batch.to(DEVICE)
            batch = Batch(src_batch, tgt_batch, pad_idx)

            # FORWARD PASS VÀ BACKWARD
            # Lưu ý: loss_compute sử dụng criterion_train (Label Smoothing)
            output_logits = model(batch.src, batch.tgt, batch.src_mask, batch.tgt_mask)
            loss_value = loss_compute(output_logits, batch.tgt_y, batch.ntokens)

            # Tích lũy trên GPU
            total_loss += loss_value
            total_tokens += batch.ntokens

            # LOGGING
            if total_tokens.item() > 0:
                avg_loss = total_loss.item() / total_tokens.item()

                progress_bar.set_postfix(
                    Loss=f"{avg_loss:.4f}",
                    PPL=f"{math.exp(avg_loss):.2f}",
                    LR=f"{optimizer.rate():.6f}"
                )

        # --- ĐÁNH GIÁ TRÊN VALIDATION SET ---
        # SỬ DỤNG CRITERION_VAL (Cross-Entropy chuẩn) để đo lường chính xác
        val_loss, val_ppl = calculate_ppl(val_loader, model, pad_idx, criterion_val)

        # ... (Phần in log và lưu checkpoint giữ nguyên)
        print(f"\n===========================================================")
        print(f"EPOCH {epoch} KẾT THÚC | TRAIN LOSS: {avg_loss:.4f} | VAL LOSS: {val_loss:.4f}")
        print(f"VAL PERPLEXITY (PPL): {val_ppl:.2f}")
        print(f"===========================================================\n")

        # LƯU CHECKPOINT (Logic tương tự)
        is_best = val_ppl < best_val_ppl
        if is_best:
            best_val_ppl = val_ppl

        save_checkpoint(model, optimizer, epoch, best_val_ppl, is_best=is_best)

        model.train()

In [ ]:
def process_data_pipeline_for_test():
    print("=== BẮT ĐẦU XỬ LÝ DỮ LIỆU (TEST MODE) ===\n")

    # -----------------------------------------
    # BƯỚC 1: LOAD DỮ LIỆU THÔ
    # -----------------------------------------
    path_en = os.path.join(DATA_DIR, 'train.en')
    path_vi = os.path.join(DATA_DIR, 'train.vi')

    raw_en = load_raw_data(path_en)
    raw_vi = load_raw_data(path_vi)

    assert len(raw_en) == len(raw_vi), "Lỗi lệch dòng!"

    # --- [QUAN TRỌNG] CẮT DỮ LIỆU ĐỂ TEST ---
    if TEST_MODE_LIMIT is not None:
        print(f"\n  CHẾ ĐỘ TEST: Đang cắt dữ liệu xuống còn {TEST_MODE_LIMIT} câu...")
        raw_en = raw_en[:TEST_MODE_LIMIT]
        raw_vi = raw_vi[:TEST_MODE_LIMIT]
    # ----------------------------------------

    print(f"-> Tổng số câu sử dụng: {len(raw_en)}")

    # -----------------------------------------
    # BƯỚC 1: CHIA TẬP TRAIN / VAL
    # -----------------------------------------
    print(f"\n--- Chia tập Train/Val (Ratio: {VAL_SPLIT_RATIO}) ---")
    en_train, en_val, vi_train, vi_val = train_test_split(
        raw_en, raw_vi, test_size=VAL_SPLIT_RATIO, random_state=SEED
    )
    print(f"-> Train: {len(en_train)} | Val: {len(en_val)}")

    # -----------------------------------------
    # BƯỚC 2: LOAD MODEL SENTENCEPIECE
    # -----------------------------------------
    print("\n--- Load SPM Models ---")
    sp_en = spm.SentencePieceProcessor(); sp_en.load(os.path.join(DATA_DIR, 'spm_en.model'))
    sp_vi = spm.SentencePieceProcessor(); sp_vi.load(os.path.join(DATA_DIR, 'spm_vi.model'))

    print(f"-> Vocab Size: {sp_en.get_piece_size()} / {sp_vi.get_piece_size()}")


    # -----------------------------------------
    # BƯỚC 3: TẠO DATASET & DATALOADER
    # -----------------------------------------
    print("\n--- Tạo DataLoader ---")
    train_dataset = TranslationDatasetSPM(en_train, vi_train, sp_en, sp_vi, max_len=MAX_SEQ_LEN)
    val_dataset = TranslationDatasetSPM(en_val, vi_val, sp_en, sp_vi, max_len=MAX_SEQ_LEN)

    collate_fn = MyCollateSPM(pad_idx=PAD_IDX)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    print(f"-> Train Batches: {len(train_loader)}")
    print(f"-> Val Batches:   {len(val_loader)}")

    return train_loader, val_loader

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import math
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import copy


def main():

    train_loader, val_loader = process_data_pipeline_for_test()
    print(f"-> Train/Val Loader đã sẵn sàng (Batch Size: {BATCH_SIZE})")


    # ----------------------------------------------------
    ## BƯỚC 1.3: KHỞI TẠO MÔ HÌNH VÀ CHUYỂN DEVICE
    # ----------------------------------------------------
    print("3. Đang khởi tạo mô hình Transformer...")

    # Gọi hàm make_model đã được định nghĩa
    model = make_model(VOCAB_SIZE_SPM, VOCAB_SIZE_SPM, N=N_LAYERS, d_model=D_MODEL, h=N_HEADS, d_ff=D_FF, dropout=DROPOUT_RATE)
    model.to(DEVICE)
    print(f"-> Mô hình đã được khởi tạo và chuyển sang {DEVICE}. Tổng tham số: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")


    # ----------------------------------------------------
    ## BƯỚC 1.4: KHỞI TẠO CÁC THÀNH PHẦN TRAINING
    # ----------------------------------------------------
    print("4. Đang thiết lập Loss Function và Optimizer...")

    # 1. Base Optimizer (AdamW được khuyến nghị)
    base_optimizer = optim.AdamW(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9, weight_decay=1e-4)

    # 2. Noam Scheduler (Wrapped Optimizer)
    optimizer = NoamOpt(model_size=D_MODEL, warmup_steps=WARMUP_STEPS, optimizer=base_optimizer)

    # 3. Criterion (Label Smoothing)
    # criterion = LabelSmoothing(size=VOCAB_SIZE_VI, padding_idx=PAD_IDX, smoothing=LABEL_SMOOTHING).to(DEVICE)
    # Do KLDivLoss hoạt động trên Log-Probs, ta dùng nn.CrossEntropyLoss cho mục đích
    # tính loss trên Validation set (VALID_CRITERION)

    # Khởi tạo Criterion mới: Dùng KLDivLoss/LabelSmoothing cho Train
    criterion_train = LabelSmoothing(size=VOCAB_SIZE_SPM, padding_idx=PAD_IDX, smoothing=LABEL_SMOOTHING).to(DEVICE)

    # Khởi tạo Criterion cho Validation (CrossEntropyLoss đơn thuần)
    criterion_val = nn.CrossEntropyLoss(ignore_index=PAD_IDX, reduction='sum').to(DEVICE)

    print(f"-> Optimizer (Noam) và Loss (Label Smoothing {LABEL_SMOOTHING}) đã sẵn sàng.")


    # ----------------------------------------------------
    ## BƯỚC 1.5: CHẠY VÒNG LẶP HUẤN LUYỆN CHÍNH
    # ----------------------------------------------------
    print("5. Bắt đầu Vòng Lặp Huấn Luyện...")

    # Chạy hàm training chính đã được sửa lỗi và tối ưu
    run_full_training_pipeline(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion_train=criterion_train, # Cần sửa lại signature của hàm training
        criterion_val=criterion_val,     # Truyền vào 2 criterion khác nhau
        optimizer=optimizer,
        pad_idx=PAD_IDX,
        n_epochs=EPOCHS,
        clip_norm=GRAD_CLIP
    )


# --- BƯỚC 2: KHỐI CHẠY CHƯƠNG TRÌNH ---
if __name__ == "__main__":
    # Đảm bảo CHECKPOINT_DIR tồn tại
    if not os.path.exists(CHECKPOINT_DIR):
        os.makedirs(CHECKPOINT_DIR)

    # Gọi hàm chính
    #main()
    print("\n--- ĐÃ HOÀN TẤT SETUP HÀM MAIN ---")
    print("Bây giờ bạn chỉ cần gọi hàm main() để bắt đầu training.")


--- ĐÃ HOÀN TẤT SETUP HÀM MAIN ---
Bây giờ bạn chỉ cần gọi hàm main() để bắt đầu training.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sentencepiece as spm
import os
import math
import copy # Vẫn cần copy và math cho việc khởi tạo model

# ==============================================================================
# 1. CẤU HÌNH & LOAD SPM (Khởi tạo các hằng số cần thiết)
# ==============================================================================


print("--- 1. LOADING SPM MODELS ---")
try:
    sp_en = spm.SentencePieceProcessor()
    sp_en.load(os.path.join(DATA_DIR, 'spm_en.model'))
    sp_vi = spm.SentencePieceProcessor()
    sp_vi.load(os.path.join(DATA_DIR, 'spm_vi.model'))

except Exception as e:
    print(f"LỖI LOAD SPM: {e}. Vui lòng kiểm tra lại đường dẫn.")
    exit()


try:
    # 1. Khởi tạo model rỗng dựa trên kiến trúc đã định nghĩa
    model = make_model(VOCAB_SIZE_SPM, VOCAB_SIZE_SPM, N=N_LAYERS, d_model=D_MODEL, h=N_HEADS).to(DEVICE)
    print("-> Đã khởi tạo cấu trúc mô hình (cần load trọng số).")
except NameError:
    print("\n--- LỖI QUAN TRỌNG ---")
    print("  Lớp hoặc hàm `make_model` CHƯA ĐƯỢC ĐỊNH NGHĨA. Vui lòng chạy lại đoạn code định nghĩa kiến trúc Transformer trước đó.")
    exit()


# ------------------------------------------------------------------------------
# 3. TẢI TRỌNG SỐ TỪ CHECKPOINT VÀ HÀM DECODE
# ------------------------------------------------------------------------------

print("\n--- 3. TẢI TRỌNG SỐ VÀ SETUP INFERENCE ---")

# 1. Tải trọng số
checkpoint_path = os.path.join(CHECKPOINT_DIR, 'latest_checkpoint.pth')

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(DEVICE)
else:
    print(f"  CẢNH BÁO: Không tìm thấy checkpoint tại {checkpoint_path}.")
    print("Model sẽ chạy với trọng số ngẫu nhiên hoặc đã được khởi tạo trước đó.")

# Hàm hỗ trợ cho Masking
def subsequent_mask(size):
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)
    return subsequent_mask == 0

def make_std_mask_decode(tgt, pad_idx):
    tgt_mask = (tgt != pad_idx).unsqueeze(-2)
    tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data).to(DEVICE)
    return tgt_mask

# Hàm dịch (Greedy Decoding)
def simple_greedy_decode(model, src, src_mask, max_len, sp_vi):
    model.eval()
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(sp_vi.bos_id()).type_as(src.data).to(DEVICE)

    for _ in range(max_len - 1):
        tgt_mask = make_std_mask_decode(ys, sp_vi.pad_id())
        out = model.decode(memory, src_mask, ys, tgt_mask)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word).to(DEVICE)], dim=1)

        if next_word == sp_vi.eos_id():
            break

    # Giải mã và trả về kết quả
    translated_ids = ys.squeeze(0).tolist()[1:]
    if sp_vi.eos_id() in translated_ids:
        translated_ids = translated_ids[:translated_ids.index(sp_vi.eos_id())]

    return sp_vi.decode_ids(translated_ids)


def beam_search_decode(model, src, src_mask, max_len, sp_vi, beam_size=12):
    model.eval()
    device = src.device

    memory = model.encode(src, src_mask)

    # (sequence, log_prob)
    beams = [(torch.tensor([[sp_vi.bos_id()]], device=device), 0.0)]
    completed = []

    for _ in range(max_len - 1):
        new_beams = []

        for seq, score in beams:
            if seq[0, -1].item() == sp_vi.eos_id():
                completed.append((seq, score))
                continue

            tgt_mask = make_std_mask_decode(seq, sp_vi.pad_id())
            out = model.decode(memory, src_mask, seq, tgt_mask)
            logits = model.generator(out[:, -1])
            log_probs = F.log_softmax(logits, dim=-1)

            topk_log_probs, topk_ids = torch.topk(log_probs, beam_size, dim=-1)

            for k in range(beam_size):
                next_id = topk_ids[0, k].item()
                next_score = score + topk_log_probs[0, k].item()

                next_seq = torch.cat(
                    [seq, torch.tensor([[next_id]], device=device)], dim=1
                )

                new_beams.append((next_seq, next_score))

        # Giữ beam tốt nhất
        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]

        if len(completed) >= beam_size:
            break

    # Nếu chưa có eos → lấy beam tốt nhất
    if len(completed) == 0:
        completed = beams

    best_seq = max(completed, key=lambda x: x[1])[0]

    # Bỏ BOS và EOS
    translated_ids = best_seq.squeeze(0).tolist()[1:]
    if sp_vi.eos_id() in translated_ids:
        translated_ids = translated_ids[:translated_ids.index(sp_vi.eos_id())]

    return sp_vi.decode_ids(translated_ids)



# ------------------------------------------------------------------------------
# 4. TEST CÂU ĐƠN GIẢN
# ------------------------------------------------------------------------------
def run_simple_test(input_sentence):
    model.eval()

    # 1. Tokenize và chuẩn bị Input
    src_ids = sp_en.encode_as_ids(input_sentence.lower())
    src_ids = [BOS_IDX] + src_ids + [EOS_IDX]
    src_tensor = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(DEVICE)
    src_mask = (src_tensor != PAD_IDX).unsqueeze(-2).to(DEVICE)

    # 2. Dịch
    with torch.no_grad():
        translation = beam_search_decode(model, src_tensor, src_mask, 100, sp_vi)

    print("-" * 50)
    print(f"EN Input: {input_sentence}")
    print(f"VI Output: {translation}")
    print("-" * 50)


print("\n--- 4. THỬ NGHIỆM DỊCH (Sau khi load model) ---")
#run_simple_test("I am happy to test the translation model.")
#run_simple_test("What is your name?")

--- 1. LOADING SPM MODELS ---
-> Đã khởi tạo cấu trúc mô hình (cần load trọng số).

--- 3. TẢI TRỌNG SỐ VÀ SETUP INFERENCE ---

--- 4. THỬ NGHIỆM DỊCH (Sau khi load model) ---


# Đánh giá

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.3 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import os, math, json
import sentencepiece as spm
import sacrebleu
import matplotlib.pyplot as plt

sp_en = spm.SentencePieceProcessor()
sp_vi = spm.SentencePieceProcessor()
sp_en.load(os.path.join(DATA_DIR, 'spm_en.model'))
sp_vi.load(os.path.join(DATA_DIR, 'spm_vi.model'))

def subsequent_mask(size):
    mask = torch.triu(torch.ones(size, size, device=DEVICE), diagonal=1).bool()
    return (~mask).unsqueeze(0)

def make_std_mask_decode(tgt, pad_idx):
    tgt_mask = (tgt != pad_idx).unsqueeze(-2)
    tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1))
    return tgt_mask


model = make_model(
    VOCAB_SIZE_SPM,
    VOCAB_SIZE_SPM,
    N=6,
    d_model=512,
    h=8
).to(DEVICE)

checkpoint = torch.load(
    os.path.join(CHECKPOINT_DIR, 'best_model.pth'),
    map_location=DEVICE
)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("Model & SentencePiece loaded successfully")




# ==============================================================================
# 4. DATASET CHO PPL
# ==============================================================================
class TestDatasetPPL(Dataset):
    def __init__(self, raw_en, raw_vi):
        self.data = list(zip(raw_en, raw_vi))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        en, vi = self.data[idx]
        src = [BOS_IDX] + sp_en.encode_as_ids(en.lower()) + [EOS_IDX]
        tgt = [BOS_IDX] + sp_vi.encode_as_ids(vi.lower()) + [EOS_IDX]
        return torch.tensor(src), torch.tensor(tgt)

class CollatePPL:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        srcs, tgts = zip(*batch)
        srcs = pad_sequence(srcs, batch_first=True, padding_value=self.pad_idx)
        tgts = pad_sequence(tgts, batch_first=True, padding_value=self.pad_idx)
        return srcs.to(DEVICE), tgts.to(DEVICE)

# ==============================================================================
# 5. PERPLEXITY
# ==============================================================================
def calculate_ppl(loader):
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX, reduction='sum')
    total_loss, total_tokens = 0, 0

    with torch.no_grad():
        for src, tgt in tqdm(loader, desc="Calculating PPL"):
            tgt_in = tgt[:, :-1]
            tgt_y = tgt[:, 1:]

            src_mask = (src != PAD_IDX).unsqueeze(-2)
            tgt_mask = make_std_mask_decode(tgt_in, PAD_IDX)

            logits = model(src, tgt_in, src_mask, tgt_mask)

            loss = criterion(
                logits.view(-1, logits.size(-1)),
                tgt_y.contiguous().view(-1)
            )

            total_loss += loss.item()
            total_tokens += (tgt_y != PAD_IDX).sum().item()

    avg_loss = total_loss / total_tokens
    return avg_loss, math.exp(avg_loss)

# ==============================================================================
# 6. BEAM SEARCH (1 câu)
# ==============================================================================
def beam_translate(src_sentence):
    src_ids = [BOS_IDX] + sp_en.encode_as_ids(src_sentence.lower()) + [EOS_IDX]
    src = torch.tensor(src_ids).unsqueeze(0).to(DEVICE)
    src_mask = (src != PAD_IDX).unsqueeze(-2)

    memory = model.encode(src, src_mask)

    beams = [(0.0, torch.tensor([[BOS_IDX]], device=DEVICE))]

    for _ in range(MAX_LEN_DECODE):
        candidates = []

        for score, seq in beams:
            if seq[0, -1].item() == EOS_IDX:
                candidates.append((score, seq))
                continue

            tgt_mask = make_std_mask_decode(seq, PAD_IDX)
            out = model.decode(memory, src_mask, seq, tgt_mask)
            log_prob = F.log_softmax(model.generator(out[:, -1]), dim=-1)

            topk = torch.topk(log_prob, BEAM_SIZE)

            for i in range(BEAM_SIZE):
                new_score = score + topk.values[0, i].item()
                new_seq = torch.cat(
                    [seq, topk.indices[0, i].view(1, 1)], dim=1
                )
                candidates.append((new_score, new_seq))

        beams = sorted(
            candidates,
            key=lambda x: x[0] / (x[1].size(1) ** 0.7),
            reverse=True
        )[:BEAM_SIZE]

        if beams[0][1][0, -1].item() == EOS_IDX:
            break

    best_seq = beams[0][1][0, 1:].tolist()
    if EOS_IDX in best_seq:
        best_seq = best_seq[:best_seq.index(EOS_IDX)]

    return sp_vi.decode_ids(best_seq)

# ==============================================================================
# 7. BLEU + REPORT
# ==============================================================================
def evaluate():
    raw_en = load_raw_data(TEST_EN_PATH)
    raw_vi = load_raw_data(TEST_VI_PATH)

    ppl_ds = TestDatasetPPL(raw_en, raw_vi)
    ppl_loader = DataLoader(
        ppl_ds,
        batch_size=BATCH_SIZE,
        collate_fn=CollatePPL(PAD_IDX)
    )

    loss, ppl = calculate_ppl(ppl_loader)

    print("\n Translating with Beam Search...")
    hypotheses = [beam_translate(s) for s in tqdm(raw_en)]

    bleu = sacrebleu.corpus_bleu(hypotheses, [raw_vi])

    print("\n" + "="*60)
    print(" FINAL EVALUATION RESULTS")
    print("="*60)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Perplexity (PPL): {ppl:.2f}")
    print(f"BLEU Score: {bleu.score:.2f}")
    print("="*60)
    print(bleu.format())
    print("="*60)

    return ppl, bleu.score

# ==============================================================================
# 8. RUN
# ==============================================================================
evaluate()


✅ Model & SentencePiece loaded successfully
-> Đang đọc file: tst2013.en...
-> Đang đọc file: tst2013.vi...


Calculating PPL: 100%|██████████| 20/20 [00:03<00:00,  5.06it/s]



🔁 Translating with Beam Search...


100%|██████████| 1268/1268 [28:43<00:00,  1.36s/it]



📊 FINAL EVALUATION RESULTS
Test Loss: 2.0334
Test Perplexity (PPL): 7.64
BLEU Score: 26.37
BLEU = 26.37 59.3/33.9/20.7/12.9 (BP = 0.975 ratio = 0.975 hyp_len = 32905 ref_len = 33738)


(7.639712025620963, 26.36685262435577)